In [1]:
# Python
import pandas as pd
import numpy as np
from fbprophet import Prophet
%matplotlib inline


In [2]:
# Python
df = pd.read_csv('./data/example_wp_peyton_manning.csv')
df['y'] = np.log(df['y'])
df.head()

In [3]:
# Python
m = Prophet()
m.fit(df);

In [4]:
# Python
future = m.make_future_dataframe(periods=365)
future.tail()

In [5]:
# Python
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [6]:
# Python
m.plot(forecast);

In [7]:
# Python
m.plot_components(forecast);
